In this notebook, we will write a complete pipeline for **training a model to obtain node embeddings**.

To begin, we will again work with the [Karate Club Network](https://en.wikipedia.org/wiki/Zachary%27s_karate_club). We will compute several statistics for this graph and then transform its structure into a PyTorch tensor.

Next, we will implement a training algorithm for graphs: a model for obtaining node embeddings.

We will also explore graph kernels.

In [ ]:
import networkx as nx

In [ ]:
G = nx.karate_club_graph()

# Visualize the graph
nx.draw(G, with_labels = True)

**Average node degree**

In [ ]:
def average_degree(num_edges, num_nodes):
    # TODO: Implement this function that takes number of edges
    # and number of nodes, and returns the average node degree of 
    # the graph. Round the result to nearest integer

    avg_degree = 0

    ############# Your code here ############

    #########################################
    
    avg_degree = round(2 * num_edges / num_nodes)

    return avg_degree

num_edges = G.number_of_edges()
num_nodes = G.number_of_nodes()
avg_degree = average_degree(num_edges, num_nodes)
print("Average degree of karate club network is {}".format(avg_degree))

__Average clustering coefficient__

In [ ]:
def average_clustering_coefficient(G):
    # TODO: Implement this function that takes a nx.Graph
    # and returns the average clustering coefficient. Round 
    # the result to 2 decimal places (for example 3.333 will
    # be rounded to 3.33 and 3.7571 will be rounded to 3.76)

    avg_cluster_coef = 0

    ############# Your code here ############
    ## Note: 
    ## 1: Please use the appropriate NetworkX clustering function

    avg_cluster_coef = nx.algorithms.approximation.clustering_coefficient.average_clustering(G)

    return avg_cluster_coef, nx.algorithms.average_clustering(G)

avg_cluster_coef = average_clustering_coefficient(G)
print("Average clustering coefficient of karate club network is {}".format(avg_cluster_coef))

__Сloseness centrality__

$c(v) = \frac{1}{\sum_{u \neq v}\text{shortest path length between } u \text{ and } v}$

In [ ]:
def closeness_centrality(G, node=5):
    # TODO: Implement the function that calculates closeness centrality 
    # for a node in karate club network. G is the input karate club 
    # network and node is the node id in the graph. Please round the 
    # closeness centrality result to 2 decimal places.

    closeness = 0

    ## Note:
    ## 1: You can use networkx closeness centrality function.
    ## 2: Notice that networkx closeness centrality returns the normalized 
    ## closeness directly, which is different from the raw (unnormalized) 
    ## one that we learned in the lecture.

    #########################################

    closeness = nx.centrality.closeness_centrality(G, node)

    return closeness

node = 5
closeness = closeness_centrality(G, node=node)
print("The node 5 has closeness centrality {}".format(closeness))

### Link prediction

__Jaccard coefficient__

In [ ]:
G = nx.complete_graph(5)

nx.draw(G)

In [ ]:
preds = nx.jaccard_coefficient(G, [(0, 1), (2, 3)])
for u, v, p in preds:
     print('(%d, %d) -> %.8f' % (u, v, p))

__Common neighbors__

In [ ]:
sorted(nx.common_neighbors(G, 0, 1))

In [ ]:
preds = nx.common_neighbor_centrality(G, [(0, 1), (2, 3)])
for u, v, p in preds:
    print(f"({u}, {v}) -> {p}")

__Adamic-Adar index__

In [ ]:
preds = nx.adamic_adar_index(G, [(0, 1), (2, 3)])
for u, v, p in preds:
    print('(%d, %d) -> %.8f' % (u, v, p))

## Converting the graph to a PyTorch tensor

In [ ]:
import torch

G = nx.karate_club_graph()

### Tensors in PyTorch

In [ ]:
# Generate 3 x 4 tensor with all ones
ones = torch.ones(3, 4)
print(ones)

# Generate 3 x 4 tensor with all zeros
zeros = torch.zeros(3, 4)
print(zeros)

# Generate 3 x 4 tensor with random values on the interval [0, 1)
random_tensor = torch.rand(3, 4)
print(random_tensor)

# Get the shape of the tensor
print(ones.shape)

In [ ]:
# Create a 3 x 4 tensor with all 32-bit floating point zeros
zeros = torch.zeros(3, 4, dtype=torch.float32)
print(zeros.dtype)

# Change the tensor dtype to 64-bit integer
zeros = zeros.type(torch.long)
print(zeros.dtype)

Retrieve the list of graph edges and convert it into a `torch.LongTensor`.

In [ ]:
def graph_to_edge_list(G):
    # TODO: Implement the function that returns the edge list of
    # an nx.Graph. The returned edge_list should be a list of tuples
    # where each tuple is a tuple representing an edge connected 
    # by two nodes.

    edge_list = list(G.edges())

    ############# Your code here ############

    #########################################

    return edge_list

def edge_list_to_tensor(edge_list):
    # TODO: Implement the function that transforms the edge_list to
    # tensor. The input edge_list is a list of tuples and the resulting
    # tensor should have the shape [2 x len(edge_list)].

    edge_index = torch.tensor([])

    ############# Your code here ############

    #########################################


    edge_index = torch.tensor(edge_list, dtype=torch.long).permute((1, 0))

    return edge_index

pos_edge_list = graph_to_edge_list(G)
pos_edge_index = edge_list_to_tensor(pos_edge_list)
print("The pos_edge_index tensor has shape {}".format(pos_edge_index.shape))

Let's sample negative edges.

In [ ]:
import random


def sample_negative_edges(G, num_neg_samples):
    # TODO: Implement the function that returns a list of negative edges.
    # The number of sampled negative edges is num_neg_samples. You do not
    # need to consider the corner case when the number of possible negative edges
    # is less than num_neg_samples. It should be ok as long as your implementation 
    # works on the karate club network. In this implementation, self loops should 
    # not be considered as either a positive or negative edge.

    neg_edge_list = []

    ############# Your code here ############

    #########################################

    pos_set = set(G.edges())
    visited_set = set()

    # G_new = random.shuffle([i for i in G.nodes()])
    
    for n_i in G.nodes():
        for n_j in G.nodes():
            if n_i == n_j or (n_i, n_j) in pos_set or (n_j, n_i) in pos_set or (n_i, n_j) in visited_set or (n_j, n_i) in visited_set:
                continue
            neg_edge_list.append((n_i, n_j))
            visited_set.add((n_i, n_j))
            visited_set.add((n_j, n_i))
            if len(visited_set) == num_neg_samples:
                break

    return neg_edge_list

# Sample 78 negative edges
neg_edge_list = sample_negative_edges(G, len(pos_edge_list))

# Transform the negative edge list to tensor
neg_edge_index = edge_list_to_tensor(neg_edge_list)
print("The neg_edge_index tensor has shape {}".format(neg_edge_index.shape))

# Which of following edges can be negative ones?
edge_1 = (7, 1)
edge_2 = (1, 33)
edge_3 = (33, 22)
edge_4 = (0, 4)
edge_5 = (4, 2)

############# Your code here ############
## Note:
## 1: For each of the 5 edges, print whether it can be negative edge

#########################################

## Training Node Embeddings

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [ ]:
emb_sample = nn.Embedding(num_embeddings=4, embedding_dim=8)
print('Sample embedding layer: {}'.format(emb_sample))

In [ ]:
# Select an embedding in emb_sample
id = torch.LongTensor([1])
print(emb_sample(id))

# Select multiple embeddings
ids = torch.LongTensor([1, 3])
print(emb_sample(ids))

# Get the shape of the embedding weight matrix
shape = emb_sample.weight.data.shape
print(shape)

# Overwrite the weight to tensor with all ones
emb_sample.weight.data = torch.ones(shape)

# Let's check if the emb is indeed initilized
ids = torch.LongTensor([0, 3])
print(emb_sample(ids))

### Creating the Node Embedding Matrix  
- We want to obtain a **16-dimensional** vector for each node in the Karate Club graph.  
- We will initialize the matrix with a **uniform distribution** in the range \([0, 1)\). This can be done using [`torch.rand`](https://pytorch.org/docs/stable/generated/torch.rand.html).

In [ ]:
# Please do not change / reset the random seed
torch.manual_seed(42)

def create_node_emb(num_node=34, embedding_dim=16):
    # TODO: Implement this function that will create the node embedding matrix.
    # A torch.nn.Embedding layer will be returned. You do not need to change 
    # the values of num_node and embedding_dim. The weight matrix of returned 
    # layer should be initialized under uniform distribution. 

    emb = torch.nn.Embedding(num_embeddings=num_node, embedding_dim=embedding_dim)

    ############# Your code here ############

    #########################################

    return emb

emb = create_node_emb()
ids = torch.LongTensor([0, 3])

# Print the embedding layer
print("Embedding: {}".format(emb))

# An example that gets the embeddings for node 0 and 3
print(emb(ids))

### Visualizing Embeddings Using PCA

In [ ]:
def visualize_emb(emb):
    X = emb.weight.data.numpy()
    pca = PCA(n_components=2)
    components = pca.fit_transform(X)
    plt.figure(figsize=(6, 6))
    club1_x = []
    club1_y = []
    club2_x = []
    club2_y = []
    for node in G.nodes(data=True):
        if node[1]['club'] == 'Mr. Hi':
            club1_x.append(components[node[0]][0])
            club1_y.append(components[node[0]][1])
        else:
            club2_x.append(components[node[0]][0])
            club2_y.append(components[node[0]][1])
    plt.scatter(club1_x, club1_y, color="red", label="Mr. Hi")
    plt.scatter(club2_x, club2_y, color="blue", label="Officer")
    plt.legend()
    plt.show()


visualize_emb(emb)

### Now Let's Move on to Training  

We want to optimize the embeddings for the task of classifying edges as positive or negative. By taking the edges and the embeddings for each node, the dot product of the embeddings, followed by a sigmoid function, should yield the probability of whether an edge is positive or negative.

In [ ]:
from torch.optim import SGD
import torch.nn as nn

def accuracy(pred, label):
    # TODO: Implement the accuracy function. This function takes the 
    # pred tensor (the resulting tensor after sigmoid) and the label 
    # tensor (torch.LongTensor). Predicted value greater than 0.5 will 
    # be classified as label 1. Else it will be classified as label 0.
    # The returned accuracy should be rounded to 4 decimal places. 
    # For example, accuracy 0.82956 will be rounded to 0.8296.

    accu = 0.0

    ############# Your code here ############

    #########################################

    accu = round(((torch.round(pred) == label).sum()/label.size(0)).item(), 4)

    return accu

def train(emb, loss_fn, sigmoid, train_label, train_edge):
    # TODO: Train the embedding layer here. You can also change epochs and 
    # learning rate. In general, you need to implement: 
    # (1) Get the embeddings of the nodes in train_edge
    # (2) Dot product the embeddings between each node pair
    # (3) Feed the dot product result into sigmoid
    # (4) Feed the sigmoid output into the loss_fn
    # (5) Print both loss and accuracy of each epoch 
    # (6) Update the embeddings using the loss and optimizer 
    # (as a sanity check, the loss should decrease during training)

    epochs = 500
    learning_rate = 0.1

    optimizer = SGD(emb.parameters(), lr=learning_rate, momentum=0.9)

    for i in range(epochs):
        optimizer.zero_grad()

        product = torch.sum(torch.mul(emb(train_edge[0]), emb(train_edge[1])), axis=1)
        pred = torch.sigmoid(product)
        loss = loss_fn(pred, train_label)
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            acc = accuracy(pred, train_label)
            if i % 100 == 0:
                visualize_emb(emb)
            print(loss.item(), acc)

    ############# Your code here ############
    
#########################################

loss_fn = nn.BCELoss()
sigmoid = nn.Sigmoid()

print(pos_edge_index.shape)

# Generate the positive and negative labels
pos_label = torch.ones(pos_edge_index.shape[1], )
neg_label = torch.zeros(neg_edge_index.shape[1], )

# Concat positive and negative labels into one tensor
train_label = torch.cat([pos_label, neg_label], dim=0)

# Concat positive and negative edges into one tensor
# Since the network is very small, we do not split the edges into val/test sets
train_edge = torch.cat([pos_edge_index, neg_edge_index], dim=1)
print(train_edge.shape)

train(emb, loss_fn, sigmoid, train_label, train_edge)

### Now, Let's Visualize the Trained Embeddings  
As we can see, the classes are now more clearly separated.

In [ ]:
visualize_emb(emb)

## Approaches with Graph Kernels

### Counting Graphlets of a Given Length

In [ ]:
g = nx.Graph()
g.add_edge(1,2);g.add_edge(1,3)
g.add_edge(1,7);g.add_edge(2,4)
g.add_edge(3,4);g.add_edge(3,5)
g.add_edge(3,6);g.add_edge(4,5)
g.add_edge(5,6);g.add_edge(6,7)

import itertools

target = nx.Graph()
target.add_edge(1,2)
target.add_edge(2,3)

for sub_nodes in itertools.combinations(g.nodes(),len(target.nodes())):
    subg = g.subgraph(sub_nodes)
    if nx.is_connected(subg) and nx.is_isomorphic(subg, target):
        print(subg.edges())

### Now, Let's Train a Model on Graph Kernels

In [ ]:
import networkx as nx
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [ ]:
# Generate simple dataset
def create_dataset():
    Gs = [nx.cycle_graph(i) for i in range(3, 103)]
    Gs.extend([nx.path_graph(i) for i in range(3, 103)])
    y = [0 if i < 100 else 1 for i in range(200)]

    return Gs, y

In [ ]:
Gs, y = create_dataset()
G_train, G_test, y_train, y_test = train_test_split(Gs, y, test_size=0.1)

In [ ]:
nx.draw(Gs[150])

In [ ]:
# Compute the shortest path kernel
def shortest_path_kernel(Gs_train, Gs_test):
    all_paths = dict()
    sp_counts_train = dict()

    for i, G in enumerate(Gs_train):
        sp_lengths = dict(nx.shortest_path_length(G))
        sp_counts_train[i] = dict()
        nodes = G.nodes()
        for v1 in nodes:
            for v2 in nodes:
                if v2 in sp_lengths[v1]:
                    length = sp_lengths[v1][v2]
                    if length in sp_counts_train[i]:
                        sp_counts_train[i][length] += 1
                    else:
                        sp_counts_train[i][length] = 1

                    if length not in all_paths:
                        all_paths[length] = len(all_paths)

    sp_counts_test = dict()

    for i, G in enumerate(Gs_test):
        sp_lengths = dict(nx.shortest_path_length(G))
        sp_counts_test[i] = dict()
        nodes = G.nodes()
        for v1 in nodes:
            for v2 in nodes:
                if v2 in sp_lengths[v1]:
                    length = sp_lengths[v1][v2]
                    if length in sp_counts_test[i]:
                        sp_counts_test[i][length] += 1
                    else:
                        sp_counts_test[i][length] = 1

                    if length not in all_paths:
                        all_paths[length] = len(all_paths)

    phi_train = np.zeros((len(G_train), len(all_paths)))
    for i in range(len(G_train)):
        for length in sp_counts_train[i]:
            phi_train[i, all_paths[length]] = sp_counts_train[i][length]

    phi_test = np.zeros((len(Gs_test), len(all_paths)))
    for i in range(len(Gs_test)):
        for length in sp_counts_test[i]:
            phi_test[i, all_paths[length]] = sp_counts_test[i][length]

    K_train = np.dot(phi_train, phi_train.T)
    K_test = np.dot(phi_test, phi_train.T)

    return K_train, K_test

In [ ]:
# Compute the graphlet kernel
def graphlet_kernel(Gs_train, Gs_test, n_samples=200):
    graphlets = [nx.Graph(), nx.Graph(), nx.Graph(), nx.Graph()]

    graphlets[0].add_nodes_from(range(3))

    graphlets[1].add_nodes_from(range(3))
    graphlets[1].add_edge(0, 1)

    graphlets[2].add_nodes_from(range(3))
    graphlets[2].add_edge(0, 1)
    graphlets[2].add_edge(1, 2)

    graphlets[3].add_nodes_from(range(3))
    graphlets[3].add_edge(0, 1)
    graphlets[3].add_edge(1, 2)
    graphlets[3].add_edge(0, 2)

    phi_train = np.zeros((len(G_train), 4))

    for i, graph in enumerate(Gs_train):
        for j in range(n_samples):
            rnd_set = np.random.choice(graph.nodes(), 3)
            sub_g = graph.subgraph(rnd_set)
            phi_train[i] += np.array([nx.is_isomorphic(g, sub_g) for g in graphlets])

    phi_test = np.zeros((len(G_test), 4))

    for i, graph in enumerate(Gs_test):
        for j in range(n_samples):
            rnd_set = np.random.choice(graph.nodes(), 3)
            sub_g = graph.subgraph(rnd_set)
            phi_test[i] += np.array([nx.is_isomorphic(g, sub_g) for g in graphlets])


    K_train = np.dot(phi_train, phi_train.T)
    K_test = np.dot(phi_test, phi_train.T)

    return K_train, K_test

In [ ]:
K_train_sp, K_test_sp = shortest_path_kernel(G_train, G_test)

K_train_gk, K_test_gk = graphlet_kernel(G_train, G_test, 500)

model = SVC(kernel='precomputed')
print("Starting Training for the Graphlet")
model.fit(K_train_gk, y_train)
y_pred = model.predict(K_test_gk)
print("Accuracy for Graphlet", accuracy_score(y_test, y_pred))

model = SVC(kernel='precomputed')
print("Starting Training for the Shortest Path")
model.fit(K_train_sp, y_train)
y_pred = model.predict(K_test_sp)
print("Accuracy for Shortest Path", accuracy_score(y_test, y_pred))